In [1]:
import pandas as pd
import yfinance as yf
from Metrics import BacktestMetrics

In [2]:
ril_nse = yf.download('RELIANCE.NS',start='2024-01-01',end='2025-01-01')
ril_nse.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
ril_nse.to_csv('ril1.csv')
ril_nse = pd.read_csv('ril1.csv')
ril_nse = ril_nse.drop(['Adj Close', 'Low','High'], axis=1)
ril_nse.columns = ['Date','N_Close','N_Open','N_Volume']
ril_nse.head()

[*********************100%***********************]  1 of 1 completed


,Date,N_Close,N_Open,N_Volume
0,2024-01-01 00:00:00+00:00,1295.125000,1290.275024,4030540
1,2024-01-02 00:00:00+00:00,1305.849976,1292.500000,7448800
2,2024-01-03 00:00:00+00:00,1291.650024,1305.000000,9037536
3,2024-01-04 00:00:00+00:00,1298.324951,1294.000000,9612778
4,2024-01-05 00:00:00+00:00,1303.849976,1301.449951,8086406


In [3]:
ril_bse = yf.download('RELIANCE.BO',start='2024-01-01',end='2025-01-01')
ril_bse.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
ril_bse.to_csv('ril2.csv')
ril_bse = pd.read_csv('ril2.csv')
ril_bse = ril_bse.drop(['Adj Close', 'Low','High'], axis=1)
ril_bse.columns = ['Date','B_Close','B_Open','B_Volume']
ril_bse.head()

[*********************100%***********************]  1 of 1 completed


,Date,B_Close,B_Open,B_Volume
0,2024-01-01 00:00:00+00:00,1294.925049,1290.525024,135282
1,2024-01-02 00:00:00+00:00,1305.449951,1293.824951,172372
2,2024-01-03 00:00:00+00:00,1291.474976,1304.050049,215002
3,2024-01-04 00:00:00+00:00,1298.699951,1294.699951,325168
4,2024-01-05 00:00:00+00:00,1303.375000,1300.074951,931030


In [4]:
df = pd.merge(ril_nse, ril_bse, on='Date', how='inner')
df.tail()

,Date,N_Close,N_Open,N_Volume,B_Close,B_Open,B_Volume
239,2024-12-24 00:00:00+00:00,1222.750000,1222.300049,6734917,1223.500000,1231.900024,268064
240,2024-12-26 00:00:00+00:00,1216.550049,1224.250000,10016178,1216.599976,1224.650024,556935
241,2024-12-27 00:00:00+00:00,1221.050049,1218.300049,7000397,1220.949951,1216.650024,653086
242,2024-12-30 00:00:00+00:00,1210.699951,1216.400024,8818766,1210.900024,1219.650024,314452
243,2024-12-31 00:00:00+00:00,1215.449951,1208.000000,6405475,1215.449951,1210.400024,444246


In [5]:
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date")

initial_balance = 100000
money = initial_balance
no_of_stk = 0
portfolio_values = [100000]

for i in range(len(df) - 1):  
    today = df.iloc[i]
    next_day = df.iloc[i + 1]
    if today["N_Close"] < today["B_Close"]:  
        no_of_stk = money // today["N_Close"]  
        left = money - (no_of_stk * today["N_Close"])
        if next_day["B_Open"]>next_day["N_Open"]:
            money = left +  (no_of_stk * next_day["B_Open"])
        else:
            money = left +  (no_of_stk * next_day["N_Open"])
        no_of_stk = 0  

    elif today["B_Close"] < today["N_Close"]:  
        no_of_stk = money // today["B_Close"]
        left = money - (no_of_stk * today["B_Close"])
        if next_day["B_Open"]<next_day["N_Open"]:
            money = left +  (no_of_stk * next_day["N_Open"])
        else:
            money = left +  (no_of_stk * next_day["B_Open"])
        no_of_stk = 0

    portfolio_values.append(money)

print(portfolio_values)

df["Portfolio_Value"] = portfolio_values
df.tail(10)


[100000, 99915.29248046875, 99881.09619140625, 100129.41931152344, 100370.04431152344, 100587.56555175781, 101122.71179199219, 101126.61560058594, 101470.51745605469, 101522.32287597656, 101888.61926269531, 101682.39782714844, 101023.80505371094, 100283.80505371094, 100944.45861816406, 101258.36218261719, 101807.46032714844, 101738.08166503906, 102594.96032714844, 103427.95349121094, 103303.84814453125, 103951.84814453125, 104443.25341796875, 104682.87841796875, 105055.47314453125, 105648.59814453125, 106254.50170898438, 106537.37670898438, 106566.17846679688, 106860.00524902344, 106357.80700683594, 106503.35180664062, 106926.35180664062, 107034.03393554688, 107174.43920898438, 107435.43920898438, 107528.51599121094, 108091.91247558594, 108214.30895996094, 108014.50720214844, 107856.11071777344, 108671.96252441406, 108871.75891113281, 108754.75891113281, 108758.35363769531, 108439.75012207031, 108439.75012207031, 109419.76977539062, 109493.76977539062, 109839.72338867188, 110411.615966

,Date,N_Close,N_Open,N_Volume,B_Close,B_Open,B_Volume,Portfolio_Value
234,2024-12-17 00:00:00+00:00,1245.300049,1261.050049,17462791,1245.099976,1262.650024,639483,149635.481567
235,2024-12-18 00:00:00+00:00,1253.250000,1240.650024,12670179,1253.050049,1243.400024,260426,149431.487427
236,2024-12-19 00:00:00+00:00,1230.449951,1239.000000,14244653,1230.599976,1239.949951,424104,147872.575806
237,2024-12-20 00:00:00+00:00,1205.300049,1224.000000,20312896,1206.000000,1229.949951,1166317,147812.575806
238,2024-12-23 00:00:00+00:00,1222.300049,1215.000000,10052824,1222.099976,1217.250000,3332773,149270.469849
239,2024-12-24 00:00:00+00:00,1222.750000,1222.300049,6734917,1223.500000,1231.900024,268064,150466.075806
240,2024-12-26 00:00:00+00:00,1216.550049,1224.250000,10016178,1216.599976,1224.650024,556935,150699.778809
241,2024-12-27 00:00:00+00:00,1221.050049,1218.300049,7000397,1220.949951,1216.650024,653086,150915.028809
242,2024-12-30 00:00:00+00:00,1210.699951,1216.400024,8818766,1210.900024,1219.650024,314452,150755.137817
243,2024-12-31 00:00:00+00:00,1215.449951,1208.000000,6405475,1215.449951,1210.400024,444246,150717.946899


In [9]:
import numpy as np


backtest= BacktestMetrics()
portfolio_values = np.array(portfolio_values)
portfolio_returns = np.diff(portfolio_values) / portfolio_values[:-1]

strategy_cagr = backtest.calculate_cagr(start_value = portfolio_values[0], end_value = portfolio_values[-1], years = 1)
strategy_sharpe = backtest.calculate_sharpe_ratio(returns = portfolio_returns)
strategy_sortino = backtest.sortino_ratio(returns=portfolio_returns)
strategy_calmar = backtest.calmar_ratio(returns=portfolio_returns)
print("STRATEGY CAGR:", strategy_cagr, "%")
print("STRATEGY SHARPE:", strategy_sharpe)
print("SORTINO RATIO:", strategy_sortino)
print("CALMAR RATIO:", strategy_calmar)

STRATEGY CAGR: 50.717946899414066 %
STRATEGY SHARPE: 0.32129686121047457
SORTINO RATIO: 7.02147991083091
CALMAR RATIO: 14.781654934321839
